# Momentum Based Trading strategy on SnP 500 stocks

In this project we use Alorithmic trading with python to implement a basic momentum based strategy for stock trading , as well as a more realisitic High Quality Momentum based strategy.

We will obtain a pandas dataframe telling us orders to put execute , in order to implement our strategies.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
import os
from scipy import stats

I will be using IEX Cloud API to get the stock data for this project. Here we initialize our Token key and import our dataset containing list of stocks that we will be working with.

In [2]:
from dotenv import load_dotenv
load_dotenv()
IEX_CLOUD_API_TOKEN = os.getenv("IEX_CLOUD_API_TOKEN")

stocks = pd.read_csv('/Users/hitengoel/Machine Learning/Algo Trading/S&P momentum/sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


Here I am testing my token and the API by getting some data about the Apple Stock.

In [3]:
symbol='AAPL'
api_url = f'https://api.iex.cloud/v1/data/core/quote/{symbol}?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

[{'avgTotalVolume': 62676971,
  'calculationPrice': 'close',
  'change': 1.73,
  'changePercent': 0.01024,
  'close': 170.73,
  'closeSource': 'official',
  'closeTime': 1709931600413,
  'companyName': 'Apple Inc',
  'currency': 'USD',
  'delayedPrice': 170.985,
  'delayedPriceTime': 1709931582504,
  'extendedChange': -0.25,
  'extendedChangePercent': -0.00146,
  'extendedPrice': 170.48,
  'extendedPriceTime': 1709945998560,
  'high': 173.7,
  'highSource': '15 minute delayed price',
  'highTime': 1709931599983,
  'iexAskPrice': 0,
  'iexAskSize': 0,
  'iexBidPrice': 0,
  'iexBidSize': 0,
  'iexClose': 170.73,
  'iexCloseTime': 1709931599409,
  'iexLastUpdated': 1709931599409,
  'iexMarketPercent': 0.014992570643982373,
  'iexOpen': 169.01,
  'iexOpenTime': 1709908200863,
  'iexRealtimePrice': 170.73,
  'iexRealtimeSize': 11,
  'iexVolume': 1143439,
  'lastTradeTime': 1709931599409,
  'latestPrice': 170.73,
  'latestSource': 'Close',
  'latestTime': 'March 8, 2024',
  'latestUpdate': 1

In [4]:
#Initializing a string with all the Tickers of our stocks in order to use this for making batch requests to the API
tick = ''
for symbol in stocks['Ticker']:
    tick = tick + symbol + ','
tick = tick[:-1]

tick

'A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA,CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD,GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,IL

Here we make batch requests to get latest price data and price data from an year ago , which we will require for our strategy.

In [5]:

cur_call = f'https://api.iex.cloud/v1/data/core/quote/{tick}?token=sk_086bb08a94fa44abb951fa07e70f7dd8'
hist_call = f'https://api.iex.cloud/v1/data/core/historical_prices/{tick}?from=2023-03-01&to=2023-03-01&token=sk_086bb08a94fa44abb951fa07e70f7dd8'
curdata = requests.get(cur_call).json()
histdata = requests.get(hist_call).json()


Here we create a new dataframe and add the relevant info into it , we shall also be using this dataframe to give the final output at the end.

In [6]:
n = len(histdata) #due to the limitations of the API , we may not get the data for all the stocks in our list.
#moving forward we shall only consider those stocks in our strategy , for whom we got the data

mycolumns1 = ['Ticker','Price','One-Year Price Return','Number of shares to buy']
final_dataframe1 = pd.DataFrame(columns = mycolumns1)
#initializing our new dataframe for storing our retrieved info and giving final output 

for i in range(n) :
    name = histdata[i]['key']
    for bunty in curdata :
        if bunty['symbol']==name :
            curr_price = float(bunty['latestPrice'])
    
    past_price = float(histdata[i]['close'])
    per_change = (curr_price-past_price)/(past_price)
    
    final_dataframe1.loc[len(final_dataframe1.index)] = [name,curr_price,per_change,'N/A'] 
    



In [7]:
final_dataframe1
# We obtain the Latest price , as well as one year price return for the stocks 
# The number of shares to buy of each stock is the final outcome we need , which shall be determined after implmenting our strategy 

,Ticker,Price,One-Year Price Return,Number of shares to buy
0,A,147.87,0.075340,N/A
1,AAL,14.68,-0.084217,N/A
2,AAP,73.08,-0.473563,N/A
3,AAPL,170.73,0.174936,N/A
4,ABBV,178.85,0.151864,N/A
...,...,...,...,...
468,YUM,139.56,0.105864,N/A
469,ZBH,126.74,0.038172,N/A
470,ZBRA,282.59,-0.065324,N/A
471,ZION,42.24,-0.162569,N/A


Now we sort the stocks based upon their one year price returns (i.e their momentum) and get the top 50 highest momentum stocks.

In [8]:
final_dataframe1.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe1 = final_dataframe1[:50]
final_dataframe1.reset_index(inplace = True) #to reset the indexing of the updated dataframe.
final_dataframe1

,index,Ticker,Price,One-Year Price Return,Number of shares to buy
0,320,NVDA,875.28,2.856199,N/A
1,29,AMD,207.39,1.648997,N/A
2,263,LLY,762.14,1.425884,N/A
3,47,AVGO,1308.72,1.204828,N/A
4,344,PHM,112.68,1.078200,N/A
5,182,GE,167.96,0.996197,N/A
6,35,ANET,273.11,0.971059,N/A
7,268,LRCX,956.65,0.948688,N/A
8,309,NFLX,604.82,0.929373,N/A
9,34,AMZN,175.35,0.902463,N/A


Creating a utility function to take the User's portfolio value as input(i.e total amount to invest)

In [9]:
'''def portfolio_input():
    global portfolio_size 
    portfolio_size = input("Enter the value of your portfolio : ")
    try :
        float(portfolio_size)
    except ValueError :
        print("Not a valid number")
        print("Please try again")
        portfolio_size = input("Enter the value of your portfolio : ")

portfolio_input()
print(portfolio_size)'''
#a function to inout portfolio size can be written as shown , here I have taken a sample portfolio of 1 million dollars
porfolio_size = 1000000

Now , as per our strategy , we divide the portfolio value equally amongst the top 50 momentum stocks.

In [10]:
portfolio_size = 1000000
position_size = float(portfolio_size)/len(final_dataframe1.index) 
#this gives us the amount to be invested in each individual stock


for i in range(len(final_dataframe1)):
    final_dataframe1.loc[i,'Number of shares to buy'] = math.floor(position_size/final_dataframe1['Price'][i])

#dividing the position size by the stock price gives us the number of shares to buy of that stock.
#floor function is used here so that we do not overshoot our portfolio value.

final_dataframe1

,index,Ticker,Price,One-Year Price Return,Number of shares to buy
0,320,NVDA,875.28,2.856199,22
1,29,AMD,207.39,1.648997,96
2,263,LLY,762.14,1.425884,26
3,47,AVGO,1308.72,1.204828,15
4,344,PHM,112.68,1.078200,177
5,182,GE,167.96,0.996197,119
6,35,ANET,273.11,0.971059,73
7,268,LRCX,956.65,0.948688,20
8,309,NFLX,604.82,0.929373,33
9,34,AMZN,175.35,0.902463,114


This gives us the final orders that we need to execute for our strategy.
However , this strategy is not that practical as it only takes into account the one year returns. This does not give an indicator of consistent momentum. It might be that the stock only gained momentum temporarily in the last month only , which would make it a risky investment. To better this strategy , we need a more detailed momentum based strategy.

#   High Quality Momentum Based Strategy

In this strategy we will also use 1 year , 6 month , 3 month and 1 month data to determine whether a stock has sustained momentum and is a good quality stock.

In [11]:
one_year_call = hist_call
six_month_call = f'https://api.iex.cloud/v1/data/core/historical_prices/{tick}?from=2023-09-01&to=2023-09-01&token=sk_086bb08a94fa44abb951fa07e70f7dd8'
three_month_call = f'https://api.iex.cloud/v1/data/core/historical_prices/{tick}?from=2023-12-01&to=2023-12-01&token=sk_086bb08a94fa44abb951fa07e70f7dd8'
one_month_call = f'https://api.iex.cloud/v1/data/core/historical_prices/{tick}?from=2024-02-02&to=2024-02-02&token=sk_086bb08a94fa44abb951fa07e70f7dd8'

one_year_data = histdata
six_month_data = requests.get(six_month_call).json()
three_month_data = requests.get(three_month_call).json()
one_month_data = requests.get(one_month_call).json()

#making calls and getting data for 1year,6month,3month and 1month time periods.


In [12]:
j = len(one_month_data)
i = len(six_month_data)
k = len(three_month_data)
l = len(one_year_data)
m = min(i,j,k,l)

#due to API limitations , we might not get the data for all the stocks in our list
#hence we will only consider the stocks for which we can get all the data, here m gives us that number

Now we initialize our new dataframe , with columns as indicated below, to save the relevant info ,  and we will also use this dataframe to give our final output.

In [13]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)




for i in range(m) :
    sname = one_month_data[i]['key']
    omprice = one_month_data[i]['close']
    for x in curdata :
        if x['symbol']==sname:
            cprice = x['latestPrice']
            break
    else :
        continue
    
    for x in six_month_data :
        if x['key']==sname:
            smprice = x['close']
            break
    else :
        continue

    for x in three_month_data :
        if x['key']==sname:
            tmprice = x['close']
            break
    else :
        continue

    for x in one_year_data :
        if x['key']==sname:
            oyprice = x['close']
            break
    else :
        continue

    per_change1y = float((cprice-oyprice)/(oyprice))
    per_change6m = float((cprice-smprice)/(smprice))
    per_change3m = float((cprice-tmprice)/(tmprice))
    per_change1m = float((cprice-omprice)/(omprice))

    hqm_dataframe.loc[len(hqm_dataframe.index)] = [sname,cprice,'N/A',per_change1y,'N/A',per_change6m,'N/A',per_change3m,'N/A',per_change1m,'N/A','N/A']
    



Now we obtain the dataframe , with the price returns for the time periods.
However we still need to determine the percentile scores and hqm score.

In [14]:
hqm_dataframe


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,147.87,N/A,0.075340,N/A,0.212944,N/A,0.148148,N/A,0.112055,N/A,N/A
1,AAL,14.68,N/A,-0.084217,N/A,0.000000,N/A,0.127496,N/A,0.008242,N/A,N/A
2,AAP,73.08,N/A,-0.473563,N/A,0.080266,N/A,0.351082,N/A,0.078035,N/A,N/A
3,AAPL,170.73,N/A,0.174936,N/A,-0.098860,N/A,-0.107247,N/A,-0.081356,N/A,N/A
4,ABBV,178.85,N/A,0.151864,N/A,0.206815,N/A,0.247124,N/A,0.060355,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
460,YUM,139.56,N/A,0.105864,N/A,0.076520,N/A,0.096050,N/A,0.083961,N/A,N/A
461,ZBH,126.74,N/A,0.038172,N/A,0.061030,N/A,0.074341,N/A,0.003325,N/A,N/A
462,ZBRA,282.59,N/A,-0.065324,N/A,0.024062,N/A,0.171503,N/A,0.146503,N/A,N/A
463,ZION,42.24,N/A,-0.162569,N/A,0.159166,N/A,0.102296,N/A,0.065322,N/A,N/A


In [15]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

#Here we use the Stats module to get the percentiles for each price return.
for row in hqm_dataframe.index :
    for tp in time_periods :
        change_col = f'{tp} Price Return'
        percentile_col = f'{tp} Return Percentile'
        hqm_dataframe.loc[row,percentile_col] = stats.percentileofscore(hqm_dataframe[change_col],hqm_dataframe.loc[row,change_col])

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,147.87,N/A,0.075340,47.096774,0.212944,75.483871,0.148148,68.387097,0.112055,87.526882,N/A
1,AAL,14.68,N/A,-0.084217,22.150538,0.000000,28.172043,0.127496,63.010753,0.008242,29.892473,N/A
2,AAP,73.08,N/A,-0.473563,1.075269,0.080266,49.892473,0.351082,98.064516,0.078035,76.774194,N/A
3,AAPL,170.73,N/A,0.174936,60.215054,-0.098860,10.752688,-0.107247,4.301075,-0.081356,4.086022,N/A
4,ABBV,178.85,N/A,0.151864,56.989247,0.206815,74.408602,0.247124,90.537634,0.060355,63.655914,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
460,YUM,139.56,N/A,0.105864,51.182796,0.076520,48.387097,0.096050,51.397849,0.083961,80.645161,N/A
461,ZBH,126.74,N/A,0.038172,41.290323,0.061030,44.516129,0.074341,44.731183,0.003325,28.387097,N/A
462,ZBRA,282.59,N/A,-0.065324,24.731183,0.024062,34.193548,0.171503,73.548387,0.146503,93.763441,N/A
463,ZION,42.24,N/A,-0.162569,12.688172,0.159166,65.806452,0.102296,53.763441,0.065322,66.451613,N/A


Now we determine the hqm score.

In [16]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])

    hqm_dataframe.loc[row,'HQM Score'] = mean(momentum_percentiles)

hqm_dataframe

#The mean of all the percentile scores gives us the hqm score.

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,147.87,N/A,0.075340,47.096774,0.212944,75.483871,0.148148,68.387097,0.112055,87.526882,69.623656
1,AAL,14.68,N/A,-0.084217,22.150538,0.000000,28.172043,0.127496,63.010753,0.008242,29.892473,35.806452
2,AAP,73.08,N/A,-0.473563,1.075269,0.080266,49.892473,0.351082,98.064516,0.078035,76.774194,56.451613
3,AAPL,170.73,N/A,0.174936,60.215054,-0.098860,10.752688,-0.107247,4.301075,-0.081356,4.086022,19.83871
4,ABBV,178.85,N/A,0.151864,56.989247,0.206815,74.408602,0.247124,90.537634,0.060355,63.655914,71.397849
...,...,...,...,...,...,...,...,...,...,...,...,...
460,YUM,139.56,N/A,0.105864,51.182796,0.076520,48.387097,0.096050,51.397849,0.083961,80.645161,57.903226
461,ZBH,126.74,N/A,0.038172,41.290323,0.061030,44.516129,0.074341,44.731183,0.003325,28.387097,39.731183
462,ZBRA,282.59,N/A,-0.065324,24.731183,0.024062,34.193548,0.171503,73.548387,0.146503,93.763441,56.55914
463,ZION,42.24,N/A,-0.162569,12.688172,0.159166,65.806452,0.102296,53.763441,0.065322,66.451613,49.677419


Now we will sort the stocks based on their hqm scores and get the top 50 stocks with the best Hqm scores.

In [17]:
hqm_dataframe.sort_values('HQM Score', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace = True,drop = True)
hqm_dataframe



,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,NVDA,875.28,N/A,2.856199,100.0,0.804366,99.784946,0.871656,100.0,0.322975,100.0,99.946237
1,GE,167.96,N/A,0.996197,98.924731,0.470238,98.27957,0.371102,98.709677,0.230116,99.784946,98.924731
2,AMD,207.39,N/A,1.648997,99.784946,0.894838,100.0,0.708460,99.784946,0.167342,95.698925,98.817204
3,AMAT,205.56,N/A,0.753027,96.55914,0.334892,91.397849,0.356026,98.27957,0.222262,99.569892,96.451613
4,LLY,762.14,N/A,1.425884,99.569892,0.368024,94.408602,0.304945,95.913978,0.141526,93.333333,95.806452
5,LRCX,956.65,N/A,0.948688,98.494624,0.362827,93.548387,0.324853,97.204301,0.140634,92.903226,95.537634
6,RL,176.03,N/A,0.506075,88.817204,0.487619,98.709677,0.317294,96.55914,0.191243,98.064516,95.537634
7,KLAC,699.21,N/A,0.848100,97.849462,0.379058,95.913978,0.273607,93.11828,0.157019,94.623656,95.376344
8,PVH,134.33,N/A,0.674938,94.83871,0.608743,99.354839,0.349237,97.849462,0.106690,86.451613,94.623656
9,MU,97.62,N/A,0.702476,95.268817,0.386845,96.774194,0.285658,94.408602,0.128816,90.967742,94.354839


In [18]:

position_size = float(portfolio_size)/len(hqm_dataframe.index)

for row in hqm_dataframe.index :
    hqm_dataframe.loc[row,'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[row,'Price'])

hqm_dataframe

#Again we have divided our portfolio equally amongst these 50 top stocks.

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,NVDA,875.28,22,2.856199,100.0,0.804366,99.784946,0.871656,100.0,0.322975,100.0,99.946237
1,GE,167.96,119,0.996197,98.924731,0.470238,98.27957,0.371102,98.709677,0.230116,99.784946,98.924731
2,AMD,207.39,96,1.648997,99.784946,0.894838,100.0,0.708460,99.784946,0.167342,95.698925,98.817204
3,AMAT,205.56,97,0.753027,96.55914,0.334892,91.397849,0.356026,98.27957,0.222262,99.569892,96.451613
4,LLY,762.14,26,1.425884,99.569892,0.368024,94.408602,0.304945,95.913978,0.141526,93.333333,95.806452
5,LRCX,956.65,20,0.948688,98.494624,0.362827,93.548387,0.324853,97.204301,0.140634,92.903226,95.537634
6,RL,176.03,113,0.506075,88.817204,0.487619,98.709677,0.317294,96.55914,0.191243,98.064516,95.537634
7,KLAC,699.21,28,0.848100,97.849462,0.379058,95.913978,0.273607,93.11828,0.157019,94.623656,95.376344
8,PVH,134.33,148,0.674938,94.83871,0.608743,99.354839,0.349237,97.849462,0.106690,86.451613,94.623656
9,MU,97.62,204,0.702476,95.268817,0.386845,96.774194,0.285658,94.408602,0.128816,90.967742,94.354839


Hence , here we obtain our final orders that we need to place in order to implement the High Quality Momentum strategy.

Now this output can be easily be produced into an excel format for a trading team to execute the orders. 
Or another API can be used to directly place these order on the stock market.


With the tools used in this project , we can implement any other strategies like Value Based Trading , also we can integrate AI and Machine Learning into our trading strategies.